Import the packages

In [25]:
import pandas as pd
import sqlite3
import numpy as np
import requests
import io

All the URLS for each of the port pages.

In [26]:
url_vbo = "https://helpcenter.veeam.com/docs/vbo365/guide/vbo_used_ports.html"
url_vbr = "https://helpcenter.veeam.com/docs/backup/vsphere/used_ports.html"
url_vbaws = "https://helpcenter.veeam.com/docs/vbaws/guide/ports.html"
url_vbaz = "https://helpcenter.veeam.com/docs/vbazure/guide/ports.html"
url_gcp = "https://helpcenter.veeam.com/docs/vbgc/guide/ports.html"
url_windows = "https://helpcenter.veeam.com/docs/agentforwindows/userguide/ports.html"
url_linux = "https://helpcenter.veeam.com/docs/agentforlinux/userguide/used_ports.html"

Create a function to handle the HTML to DataFrame conversion.

In [27]:
def html_to_dataframe(url, product):
    html = requests.get(url)
    df = pd.read_html(io.StringIO(html.text), encoding='utf-8')
    df = pd.concat(df)
    df['Product'] = product
    return df

Run the function on all the URLs.

In [28]:
df_vbo = html_to_dataframe(url_vbo, "VB365")
df_vbr = html_to_dataframe(url_vbr, "VBR")
df_vbaws = html_to_dataframe(url_vbaws, "VBAWS")
df_vbaz = html_to_dataframe(url_vbaz, "VBAZ")
df_gcp = html_to_dataframe(url_gcp, "VBGCP")
df_windows = html_to_dataframe(url_windows, "Windows")
df_linux = html_to_dataframe(url_linux, "Linux")

Concatinate all the DataFrames into a single DataFrame

In [29]:
combined_df = pd.concat([df_vbo, df_vbr, df_vbaws, df_vbaz, df_gcp, df_windows, df_linux], ignore_index=True)

Add a product column

In [30]:
combined_df['Description'] = np.where(combined_df['Notes'].notna(), combined_df['Notes'], combined_df['Description'])

Remove the first and last column

In [31]:
combined_df.drop(columns=[0, combined_df.columns[-1]], inplace=True)

Drop all the NaN rows

In [32]:
combined_df.dropna(inplace=True)

Remove some unneeded rows

In [33]:
combined_df = combined_df[combined_df['To'] != 'Other Communications' ]
combined_df = combined_df[combined_df['To'] != 'Communication with Virtualization Servers' ]
combined_df = combined_df[combined_df['To'] != 'Communication with Backup Server' ]

Replace some hex values with their actual symbol.

In [34]:
combined_df.replace('â\x80\x94', '—', regex=True, inplace=True)

In [35]:
combined_df.replace('Â\xa0', ' ', regex=True, inplace=True)

Check they have been removed

In [36]:
len(combined_df[combined_df['Description'].str.contains(r'Â\xa0') == True])

0

In [37]:
len(combined_df[combined_df['Description'].str.contains(r'Â\xa0BackupÂ\xa0&Â\xa0') == True])

0

In [38]:
len(combined_df[combined_df['To'].str.contains(r'Â\xa0BackupÂ\xa0&Â\xa0') == True])

0

Create the database connection

In [39]:
con = sqlite3.connect("allports.db")

In [40]:
cur = con.cursor()

Create the table

In [41]:
cur.execute("CREATE TABLE IF NOT EXISTS all_ports(from_port TEXT, to_port TEXT, protocol TEXT, port TEXT, Description TEXT, Product TEXT)")

In [42]:
for row in combined_df.itertuples(index=False):
  cur.execute("INSERT INTO all_ports VALUES(?, ?, ?, ?, ?, ?)", (str(row[0]).capitalize(), str(row[1]).capitalize(), str(row[2]), str(row[3]), str(row[4]), str(row[5])))
con.commit()

Run some checks to ensure it is all working as expected.

In [43]:
cur.execute("SELECT DISTINCT product FROM all_ports")
res = cur.fetchall()
for r in res:
  print(r[0])

VB365
VBR
VBAWS
VBAZ
VBGCP
Windows
Linux


In [44]:
cur.execute("SELECT DISTINCT port FROM all_ports WHERE product = 'VB365' AND from_port = 'Backup proxy server1' AND protocol = 'TCP'")
res = cur.fetchall()
for i in res:
  print(i[0])

9191
9193
443
80 or 443
25 or 465 or 587
22
5432 (used by default)
4222 (used by default)


In [45]:
cur.execute("SELECT COUNT(*)FROM all_ports WHERE Product = 'Linux'")
res = cur.fetchall()
res[0][0]

19